# LELA32051 Computational Linguistics Week 7

This week we are going to take a look at chatbots.

## Rule-based chatbot: Eliza

As described in this week's lecture, Eliza is a chatbot that simulates a Rogerian therapist, making use of a set of rules in the form of regular expressions. At the heart of Eliza are the substitution function (re.sub in Python) and grouping. We'll start with a quick recap as to what these are.

We need to import the Regular Expressions module in Python.

In [ ]:
import re

This gives as access to the very useful function re.sub.

### re.sub()

This finds all occurences of a given sequence and replaces it with a sequence provided:

In [ ]:
utt = 'walked'
re.sub('ed','ing',utt)

### Groups

Grouping is a very powerful technique for picking out substrings from a string that matches a specified pattern. Parentheses are used to indicate the start and end of the substring. It is very powerful when combined with substitution.

You can use parentheses to capture a particular substring within a pattern and then use it in your replacement string within sub. For example:


In [ ]:
utt = "procrastinating"
re.sub('([a-z]+)ing','\\1ed',utt)

## A very simple Elizabot

The code below implements a very simple Eliza. The function respond takes an utterance as input and using re.sub to generate responses. The loop below the function creates a simple interface that takes user input and prints the response.

We can extend Eliza's ability by adding additional rules.

In [ ]:
def respond(utt):
  utt = re.sub('hello my name is (.+)','hello \\1 how are you feeling today', utt)
  return utt

In [ ]:
utt = ""
while utt != 'goodbye':
    utt = input('> ')
    reply = respond(utt)
    if reply != utt:
        print(reply)
    else:
        if utt != "goodbye":
            print("Can you rephrase that?")

### Activity

Add patterns (using substitutions and grouping) to the respond function that will allow Eliza to conduct both of these conversations. Test your system by conducting the conversation with Eliza.

User: hello my name is emma <br>
Eliza: Hello emma my name is Eliza. How are you feeling today? <br>
User: i am feeling very happy <br>
Eliza: Do you often feel happy? <br>
User: yes since I started my new job <br>
Eliza: Can you tell me about starting your new job? <br>

User: hello my name is john <br>
Eliza: Hello john, my name is Eliza. How are you feeling today? <br>
User: i am feeling pretty happy <br>
Eliza: Do you often feel happy? <br>
User: yes since I moved house <br>
Eliza: Can you tell me about moving house? <br>

### Reverse engineering the NLTK Eliza

There have been many implementations of Eliza over the years. One version is built into the NLTK toolkit. This can be run as follows:


In [ ]:
import nltk

In [ ]:
nltk.chat.eliza.demo()

Activity: Conduct a four-turn-each conversation of your own with the NLTK Eliza. Adds the substitution that you think Eliza is using to generate the responses to your own chatbot using the code below. Where you find Eliza's response to be lacking, update the substitution to give a better response.

In [ ]:
def respond(utt):
  utt = re.sub('hello my name is (.+)','hello \\1 how are you feeling today', utt)
  return utt

In [ ]:
utt = ""
while utt != 'goodbye':
    utt = input('> ')
    reply = respond(utt)
    if reply != utt:
        print(reply)
    else:
        if utt != "goodbye":
            print("Can you rephrase that?")

## Corpus-based chatbots

Training and running a corpus-based chatbot takes more steps than we have time for today. If you want to have a go in your own time, then you will find a tutorial for doing so in Pytorch here:

https://pytorch.org/tutorials/beginner/chatbot_tutorial.html

There is a link at the top of the page to open the notebook in Colab.

To run this in colab easily just put the following code block at the top and run it before working through the rest of the notebook.

In [ ]:
! wget https://zissou.infosci.cornell.edu/convokit/datasets/movie-corpus/movie-corpus.zip
! unzip movie-corpus.zip
! mkdir data
! mv movie-corpus data/